In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision as tv
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
class Dataset2class(torch.utils.data.Dataset):
    def __init__(self, path_dir1, path_dir2):
        super().__init__()

        self.path_dir1 = path_dir1
        self.path_dir2 = path_dir2

        self.dir1_list = sorted(os.listdir(path_dir1))
        self.dir2_list = sorted(os.listdir(path_dir2))

    def __len__(self):
        return len(self.dir1_list) + len(self.dir2_list)

    def __getitem__(self, idx):
        idx = idx % self.__len__()

        if idx < len(self.dir1_list):
            class_id = 0
            img_path = os.path.join(self.path_dir1, self.dir1_list[idx])
        else:
            class_id = 1
            idx -= len(self.dir1_list)
            img_path = os.path.join(self.path_dir2, self.dir2_list[idx])

        img = cv2.imread(img_path, cv2.IMREAD_COLOR)

        while img is None or img.size == 0:
            idx = (idx + 1) % self.__len__()
            if idx < len(self.dir1_list):
                class_id = 0
                img_path = os.path.join(self.path_dir1, self.dir1_list[idx])
            else:
                class_id = 1
                idx -= len(self.dir1_list)
                img_path = os.path.join(self.path_dir2, self.dir2_list[idx])
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)
            if idx == 0:
                raise IndexError("All images are empty.")

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype(np.float32)
        img = img / 255.0

        img = cv2.resize(img, (50, 50), interpolation=cv2.INTER_AREA)
        img = img.transpose((2, 0, 1))
        t_img = torch.from_numpy(img)
        t_class_id = torch.tensor(class_id)
        return {'img': t_img, 'label': t_class_id}

In [ ]:

train_cats_path = "/content/gdrive/My Drive/Colab Notebooks/dataset/train/Cats/"
train_dogs_path = "/content/gdrive/My Drive/Colab Notebooks/dataset/train/Dogs/"
test_cats_path = "/content/gdrive/My Drive/Colab Notebooks/dataset/test/Cats/"
test_dogs_path = "/content/gdrive/My Drive/Colab Notebooks/dataset/test/Dogs/"

train_ds_catsdogs = Dataset2class(train_dogs_path, train_cats_path)
test_ds_catsdogs = Dataset2class(test_dogs_path, test_cats_path)



In [ ]:
batch_size = 16

train_loader = torch.utils.data.DataLoader(
    train_ds_catsdogs, shuffle=True, drop_last=True,
    batch_size=batch_size, num_workers=0)

test_loader = torch.utils.data.DataLoader(
    test_ds_catsdogs, shuffle=True, drop_last=False,
    batch_size=batch_size, num_workers=0)

In [ ]:
model = tv.models.resnet34(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # Modify the final layer to match our number of classes

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:01<00:00, 82.1MB/s]


In [ ]:
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for batch in tqdm(train_loader):
        inputs, labels = batch['img'].to(device), batch['label'].to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

# Validation function
def validate(model, test_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in tqdm(test_loader):
            inputs, labels = batch['img'].to(device), batch['label'].to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(test_loader)
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

# Training loop
num_epochs = 10
train_losses = []
train_accuracies = []

for epoch in range(num_epochs):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")

# Save the trained model
torch.save(model.state_dict(), "cats_dogs_resnet34.pth")

# Testing loop
test_losses = []
test_accuracies = []

# Load the trained model for testing
model.load_state_dict(torch.load("cats_dogs_resnet34.pth"))

test_loss, test_acc = validate(model, test_loader, criterion, device)
test_losses.append(test_loss)
test_accuracies.append(test_acc)

print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")

100%|██████████| 593/593 [39:48<00:00,  4.03s/it]


Epoch 1/10
Train Loss: 0.5367, Train Acc: 0.7484


100%|██████████| 593/593 [11:31<00:00,  1.17s/it]


Epoch 2/10
Train Loss: 0.3884, Train Acc: 0.8294


100%|██████████| 593/593 [11:25<00:00,  1.16s/it]


...

100%|██████████| 593/593 [11:30<00:00,  1.16s/it]


Epoch 5/10
Train Loss: 0.2693, Train Acc: 0.8883


100%|██████████| 593/593 [11:25<00:00,  1.16s/it]


Epoch 6/10
Train Loss: 0.2683, Train Acc: 0.8938


 63%|██████▎   | 376/593 [07:10<03:57,  1.09s/it]